# **Amazing International Airlines Inc. (AIAI)**

## **Context**

Amazing International Airlines Inc. (AIAI) is a global airline seeking to enhance its customer loyalty program through **data-driven personalization**.  
In an increasingly competitive aviation market, AIAI aims to leverage **advanced segmentation analytics** to design tailored services and marketing strategies for its diverse customer base.

Your task is to analyze **three years of loyalty and flight activity data** to uncover meaningful customer segments that can guide business decisions and strategic initiatives.  

The segmentation should integrate multiple analytical perspectives, including:

- **Value-based segmentation** — grouping customers by their economic contribution and loyalty value.  
- **Behavioral segmentation** — analyzing flight frequency, point redemption habits, and travel companions.  
- **Demographic segmentation** — considering personal attributes such as gender, income, education, and location to reveal distinctive travel profiles.  

Ultimately, the objective is to **combine these perspectives into a unified segmentation framework**, supporting AIAI in creating differentiated experiences and maximizing long-term customer value.


## **Expected Outcomes**

In this notebook, we will perform a comprehensive **Exploratory Data Analysis (EDA)** aligned with the CRISP-DM methodology, focusing on:

- Descriptive statistics and visualization of key variables.  
- Identification of data quality issues and potential biases.  
- Detection of trends, patterns, and anomalies across customer behaviors.  
- Development of engineered features that may enhance future clustering performance.  
- Clear communication of insights to support strategic decision-making.

## **Data Definition**



### Dataset 1: `DM_AIAI_CustomerDB.csv`

| **Variable** | **Description** |
|---------------|-----------------|
| Loyalty# | Unique customer identifier for loyalty program members |
| First Name | Customer’s first name |
| Last Name | Customer’s last name |
| Customer Name | Customer’s full name (concatenated) |
| Country | Customer’s country of residence |
| Province or State | Customer’s province or state |
| City | Customer’s city of residence |
| Latitude | Geographic latitude coordinate of customer location |
| Longitude | Geographic longitude coordinate of customer location |
| Postal code | Customer’s postal/ZIP code |
| Gender | Customer’s gender |
| Education | Customer’s highest education level |
| Location Code | Urban/Suburban/Rural classification of customer residence |
| Income | Customer’s annual income |
| Marital Status | Customer’s marital status |
| LoyaltyStatus | Current tier status in loyalty program (Star > Nova > Aurora) |
| EnrollmentDateOpening | Date when customer joined the loyalty program |
| CancellationDate | Date when customer left the program |
| Customer Lifetime Value | Total calculated monetary value of customer relationship |
| EnrollmentType | Method of joining loyalty program |

### Dataset 2: `DM_AIAI_FlightsDB.csv`

| **Variable** | **Description** |
|---------------|-----------------|
| Loyalty# | Unique customer identifier linking to CustomerDB |
| Year | Year of flight activity record |
| Month | Month of flight activity record (1–12) |
| YearMonthDate | First day of the month for the activity period |
| NumFlights | Total number of flights taken by customer in the month |
| NumFlightsWithCompanions | Number of flights where customer traveled with companions |
| DistanceKM | Total distance traveled in kilometers for the month |
| PointsAccumulated | Loyalty points earned by customer during the month |
| PointsRedeemed | Loyalty points spent/redeemed by customer during the month |
| DollarCostPointsRedeemed | Dollar value of points redeemed during the month |





### **Methodological Note**

This segmentation initiative originates from a **pedagogical impetus embedded in data-centric inquiry**, aligning technical exploration with **business-oriented insight generation**.  
All analyses are conducted under the **CRISP-DM framework**, progressing from Business Understanding and Data Understanding toward Modeling and Evaluation.

---

# **Libraries and Functions**

This section imports the essential Python libraries and defines reusable functions that will support the exploratory data analysis (EDA) process for the **Amazing International Airlines Inc. (AIAI)** segmentation project.  
All functions are designed for clarity, reproducibility, and consistency across the analysis workflow.

## **Import necessary libraries**


In [1380]:
#%load_ext nb_black

# Library to suppress warnings or deprecation notes
import warnings

warnings.filterwarnings("ignore")

# Libraries to help with reading and manipulating data

import pandas as pd
import numpy as np
import os

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.impute import KNNImputer
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.manifold import TSNE

# libaries to help with data visualization
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder
import datetime
from sklearn.impute import KNNImputer 
import plotly.express as px
from sklearn.neighbors import LocalOutlierFactor
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from scipy import stats
import matplotlib.cm as cm
from sklearn import metrics
from sklearn.metrics import make_scorer, silhouette_score, silhouette_samples, davies_bouldin_score, calinski_harabasz_score
from scipy.cluster.hierarchy import dendrogram
from sklearn.model_selection import GridSearchCV

## **Functions**

### Files to be loaded:
- `DM_AIAI_CustomerDB.csv` — customer-level information from the loyalty program  
- `DM_AIAI_FlightsDB.csv` — monthly flight activity data linked via the `Loyalty#` key  

### Objective:
Load both datasets from the local working directory into pandas DataFrames for subsequent cleaning, exploration, and feature engineering.


In [1381]:
# Define file paths (adjust if your folder structure changes)
customer_path = '/Users/zemtiago/Desktop/Projeto DM 2025/Project Guidelines-20251010/data/DM_AIAI_CustomerDB.csv'
flights_path  = '/Users/zemtiago/Desktop/Projeto DM 2025/Project Guidelines-20251010/data/DM_AIAI_FlightsDB.csv'

# Load the CSV files into pandas DataFrames
df_customer = pd.read_csv(customer_path)
df_flights  = pd.read_csv(flights_path)

print("Data successfully loaded\n")

Data successfully loaded



In [1382]:
# Display confirmation and basic info
print(f"Customer dataset: {df_customer.shape[0]} rows × {df_customer.shape[1]} columns")
print(f"Flights dataset:  {df_flights.shape[0]} rows × {df_flights.shape[1]} columns")

# Optional: preview the first few rows
display(df_customer.head(3))
display(df_flights.head(3))

Customer dataset: 16921 rows × 21 columns
Flights dataset:  608436 rows × 10 columns


,Unnamed: 0,Loyalty#,First Name,Last Name,Customer Name,Country,Province or State,City,Latitude,Longitude,Postal code,Gender,Education,Location Code,Income,Marital Status,LoyaltyStatus,EnrollmentDateOpening,CancellationDate,Customer Lifetime Value,EnrollmentType
0,0,480934,Cecilia,Householder,Cecilia Householder,Canada,Ontario,Toronto,43.653225,-79.383186,M2Z 4K1,female,Bachelor,Urban,70146.0,Married,Star,2/15/2019,NaN,3839.14,Standard
1,1,549612,Dayle,Menez,Dayle Menez,Canada,Alberta,Edmonton,53.544388,-113.490930,T3G 6Y6,male,College,Rural,0.0,Divorced,Star,3/9/2019,NaN,3839.61,Standard
2,2,429460,Necole,Hannon,Necole Hannon,Canada,British Columbia,Vancouver,49.282730,-123.120740,V6E 3D9,male,College,Urban,0.0,Single,Star,7/14/2017,1/8/2021,3839.75,Standard


,Loyalty#,Year,Month,YearMonthDate,NumFlights,NumFlightsWithCompanions,DistanceKM,PointsAccumulated,PointsRedeemed,DollarCostPointsRedeemed
0,413052,2021,12,12/1/2021,2.0,2.0,9384.0,938.0,0.0,0.0
1,464105,2021,12,12/1/2021,0.0,0.0,0.0,0.0,0.0,0.0
2,681785,2021,12,12/1/2021,10.0,3.0,14745.0,1474.0,0.0,0.0


# **Exploratory Data Analysis**

This section focuses on the **Data Understanding** phase of the CRISP-DM framework.  
We explore the structure, completeness, and consistency of both datasets — `CustomerDB` and `FlightsDB` — to assess their readiness for clustering and feature engineering.


## Dataframe structure and datatypes

In this section, we inspect the structure and data types of the two AIAI datasets:  
- `df_customer`: loyalty program and demographic information.  
- `df_flights`: monthly flight activity data.

This step helps us confirm that the data was imported correctly, identify column types, and detect potential inconsistencies (e.g., numeric fields stored as objects).

In [1383]:
print("=== Customer Dataset Info ===")
df_customer.info()


=== Customer Dataset Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16921 entries, 0 to 16920
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               16921 non-null  int64  
 1   Loyalty#                 16921 non-null  int64  
 2   First Name               16921 non-null  object 
 3   Last Name                16921 non-null  object 
 4   Customer Name            16921 non-null  object 
 5   Country                  16921 non-null  object 
 6   Province or State        16921 non-null  object 
 7   City                     16921 non-null  object 
 8   Latitude                 16921 non-null  float64
 9   Longitude                16921 non-null  float64
 10  Postal code              16921 non-null  object 
 11  Gender                   16921 non-null  object 
 12  Education                16921 non-null  object 
 13  Location Code            16921 non-null  objec

In [1384]:
print("\n=== Flights Dataset Info ===")
df_flights.info()


=== Flights Dataset Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 608436 entries, 0 to 608435
Data columns (total 10 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Loyalty#                  608436 non-null  int64  
 1   Year                      608436 non-null  int64  
 2   Month                     608436 non-null  int64  
 3   YearMonthDate             608436 non-null  object 
 4   NumFlights                608436 non-null  float64
 5   NumFlightsWithCompanions  608436 non-null  float64
 6   DistanceKM                608436 non-null  float64
 7   PointsAccumulated         608436 non-null  float64
 8   PointsRedeemed            608436 non-null  float64
 9   DollarCostPointsRedeemed  608436 non-null  float64
dtypes: float64(6), int64(3), object(1)
memory usage: 46.4+ MB


In [1385]:
# Creating a copy of ABCDEats to keep the original data intact
data_customer = df_customer.copy()
data_flights = df_flights.copy()

# Basic shapes
print(f"Customer dataset shape: {data_customer.shape}")
print(f"Flights dataset shape:  {data_flights.shape}")

Customer dataset shape: (16921, 21)
Flights dataset shape:  (608436, 10)


In [1386]:
# View random records to inspect data values and formatting
display(data_customer.sample(n=5, random_state=42))


,Unnamed: 0,Loyalty#,First Name,Last Name,Customer Name,Country,Province or State,City,Latitude,Longitude,Postal code,Gender,Education,Location Code,Income,Marital Status,LoyaltyStatus,EnrollmentDateOpening,CancellationDate,Customer Lifetime Value,EnrollmentType
8444,8444,672039,Deonna,Tasson,Deonna Tasson,Canada,Ontario,Ottawa,45.421532,-75.697189,K1F 2R2,male,Bachelor,Suburban,83761.0,Married,Nova,8/8/2015,NaN,8945.69,Standard
15778,15778,123668,Josef,Krausz,Josef Krausz,Canada,Ontario,Sudbury,46.522099,-80.953033,M5V 1G5,male,Doctor,Suburban,24392.0,Married,Star,9/23/2017,NaN,10387.25,Standard
169,169,553936,Tianna,Kozakiewicz,Tianna Kozakiewicz,Canada,Ontario,Toronto,43.653225,-79.383186,P1J 8T7,female,Bachelor,Rural,29462.0,Single,Star,12/11/2017,8/11/2018,4170.69,Standard
2919,2919,854835,Nanette,Schieffer,Nanette Schieffer,Canada,Ontario,Trenton,44.101128,-77.576309,K8V 4B2,male,Bachelor,Suburban,60613.0,Single,Aurora,6/29/2019,2/28/2020,9366.82,Standard
11136,11136,492496,Mickie,Koppenhaver,Mickie Koppenhaver,Canada,British Columbia,Vancouver,49.282730,-123.120740,V6E 3Z3,female,Bachelor,Rural,23867.0,Married,Star,4/22/2020,NaN,2556.61,Standard


In [1387]:
# View random records to inspect data values and formatting
display(data_flights.sample(n=5, random_state=42))

,Loyalty#,Year,Month,YearMonthDate,NumFlights,NumFlightsWithCompanions,DistanceKM,PointsAccumulated,PointsRedeemed,DollarCostPointsRedeemed
24268,600220,2020,4,4/1/2020,0.0,0.0,0.0,0.00,0.0,0.0
366190,644945,2020,9,9/1/2020,3.0,3.0,22549.0,2254.00,3717.0,37.0
392767,550262,2020,7,7/1/2020,0.0,0.0,0.0,0.00,0.0,0.0
243532,112863,2021,3,3/1/2021,0.0,0.0,0.0,0.00,0.0,0.0
580072,391542,2019,11,11/1/2019,7.2,0.9,17976.6,1797.66,0.0,0.0


## Missing Values

In [1388]:
data_customer.isnull().sum()


Unnamed: 0                     0
Loyalty#                       0
First Name                     0
Last Name                      0
Customer Name                  0
Country                        0
Province or State              0
City                           0
Latitude                       0
Longitude                      0
Postal code                    0
Gender                         0
Education                      0
Location Code                  0
Income                        20
Marital Status                 0
LoyaltyStatus                  0
EnrollmentDateOpening          0
CancellationDate           14611
Customer Lifetime Value       20
EnrollmentType                 0
dtype: int64

In [1389]:
data_flights.isnull().sum()


Loyalty#                    0
Year                        0
Month                       0
YearMonthDate               0
NumFlights                  0
NumFlightsWithCompanions    0
DistanceKM                  0
PointsAccumulated           0
PointsRedeemed              0
DollarCostPointsRedeemed    0
dtype: int64

In [1390]:
n_total = len(data_customer)
n_missing = data_customer['CancellationDate'].isna().sum()
print(f"Missing: {n_missing}  ({n_missing/n_total:.2%})")


Missing: 14611  (86.35%)


The variable CancellationDate contains approximately 86% missing values. This is expected, as missing entries correspond to active loyalty members who have not left the program. Only around 14% of customers show a valid cancellation date, indicating a relatively low churn rate across all loyalty tiers (Aurora, Nova, Star).

### **Income**

In [1391]:
df_customer['Income'].isna().sum(), data_customer['Income'].isna().mean()*100


(20, 0.11819632409432067)

In [1392]:
import pandas as pd
import numpy as np

# Garantir que Income é numérico (coerce converte texto inválido em NaN)
data_customer['Income'] = pd.to_numeric(data_customer['Income'], errors='coerce')

# Contagens básicas
n_total = len(data_customer)
n_nan   = data_customer['Income'].isna().sum()
n_zero  = (data_customer['Income'] == 0).sum()

print(f"Total rows: {n_total}")
print(f"NaN Income: {n_nan}  ({n_nan/n_total:.2%})")
print(f"Zero Income: {n_zero}  ({n_zero/n_total:.2%})")

# Que percentagem de NÃO-NAN é zero?
valid = n_total - n_nan
if valid > 0:
    print(f"Zero among valid (non-NaN): {n_zero}  ({n_zero/valid:.2%})")

# Quebrar por LoyaltyStatus (opcional)
ct_tier = pd.crosstab(data_customer['LoyaltyStatus'], data_customer['Income'].eq(0))
ct_tier.columns = ['Income!=0', 'Income==0']
display(ct_tier)
display((ct_tier.div(ct_tier.sum(axis=1), axis=0) * 100).round(2))  # percentagens por tier

# Top países com mais zeros (opcional)
zero_by_country = (
    df_customer.loc[data_customer['Income'].eq(0)]
    .groupby('Country')
    .size()
    .sort_values(ascending=False)
    .head(10)
)
display(zero_by_country.to_frame('zero_count'))

# Sanidade extra: negativos (devem ser raros/improváveis)
negatives = (data_customer['Income'] < 0).sum()
print(f"Negative incomes: {negatives}")


Total rows: 16921
NaN Income: 20  (0.12%)
Zero Income: 4273  (25.25%)
Zero among valid (non-NaN): 4273  (25.28%)


,Income!=0,Income==0
LoyaltyStatus,,
Aurora,2643,795
Nova,4377,1345
Star,5628,2133


,Income!=0,Income==0
LoyaltyStatus,,
Aurora,76.88,23.12
Nova,76.49,23.51
Star,72.52,27.48


,zero_count
Country,
Canada,4273


Negative incomes: 0


In [1393]:
data_customer[data_customer['Income'].isna()] \
    .groupby('Country')['Income'] \
    .size() \
    .sort_values(ascending=False)


Country
Canada    20
Name: Income, dtype: int64

In [1394]:
# Replace NaN incomes with 0, but only for Canada
data_customer.loc[
    (data_customer['Country'] == 'Canada') & (data_customer['Income'].isna()),
    'Income'
] = 0

# Sanity check
data_customer['Income'].isna().sum(), (data_customer['Income'] == 0).sum()


(0, 4293)

In [1395]:
data_customer['Income']

0        70146.0
1            0.0
2            0.0
3            0.0
4        97832.0
5        26262.0
6            0.0
7        93272.0
8        93272.0
9        93272.0
10       47199.0
11       24204.0
12       79628.0
13           0.0
14           0.0
15       28142.0
16       28142.0
17       84981.0
18           0.0
19           0.0
20           0.0
21           0.0
22           0.0
23           0.0
24       78108.0
25       96335.0
26       91182.0
27       91182.0
28       43578.0
29       31409.0
30           0.0
31       26605.0
32       52361.0
33       10853.0
34       60068.0
35       51833.0
36       60230.0
37       60230.0
38           0.0
39       30821.0
40           0.0
41           0.0
42           0.0
43       66676.0
44       66676.0
45       66676.0
46       66676.0
47       66676.0
48           0.0
49       62445.0
50       62445.0
51       56480.0
52       25927.0
53       20920.0
54       84229.0
55       19245.0
56       60021.0
57       60021.0
58       60021

In [1396]:
data_customer.isnull().sum()


Unnamed: 0                     0
Loyalty#                       0
First Name                     0
Last Name                      0
Customer Name                  0
Country                        0
Province or State              0
City                           0
Latitude                       0
Longitude                      0
Postal code                    0
Gender                         0
Education                      0
Location Code                  0
Income                         0
Marital Status                 0
LoyaltyStatus                  0
EnrollmentDateOpening          0
CancellationDate           14611
Customer Lifetime Value       20
EnrollmentType                 0
dtype: int64

### **CancellationDate**

In [1397]:
data_customer['CancellationDate'] = pd.to_datetime(data_customer['CancellationDate'], errors='coerce')
data_customer['CancellationDate'] = data_customer['CancellationDate'].dt.strftime('%d/%m/%Y').fillna('Active')


In [1398]:
data_customer['CancellationDate'] = data_customer['CancellationDate'].fillna('Active')

In [1399]:
data_customer['CancellationDate']

0            Active
1            Active
2        08/01/2021
3            Active
4            Active
5            Active
6            Active
7            Active
8            Active
9            Active
10           Active
11           Active
12           Active
13           Active
14           Active
15           Active
16           Active
17           Active
18           Active
19       02/01/2021
20           Active
21           Active
22           Active
23           Active
24           Active
25           Active
26           Active
27           Active
28           Active
29           Active
30           Active
31       06/12/2018
32           Active
33           Active
34           Active
35           Active
36           Active
37           Active
38           Active
39       18/04/2020
40           Active
41           Active
42           Active
43           Active
44           Active
45       03/02/2020
46           Active
47           Active
48           Active
49           Active


In [1400]:
data_customer.isnull().sum()


Unnamed: 0                  0
Loyalty#                    0
First Name                  0
Last Name                   0
Customer Name               0
Country                     0
Province or State           0
City                        0
Latitude                    0
Longitude                   0
Postal code                 0
Gender                      0
Education                   0
Location Code               0
Income                      0
Marital Status              0
LoyaltyStatus               0
EnrollmentDateOpening       0
CancellationDate            0
Customer Lifetime Value    20
EnrollmentType              0
dtype: int64

### **Customer Lifetime Value**

In [1401]:
# Normalize keys and basic setup

COL_CLV = 'Customer Lifetime Value'

data_customer['Loyalty#'] = data_customer['Loyalty#'].astype(str).str.strip()
data_flights['Loyalty#']  = data_flights['Loyalty#'].astype(str).str.strip()

In [1402]:
# Find customers with no flights

ids_in_flights = set(data_flights['Loyalty#'].unique())
mask_no_flights = ~data_customer['Loyalty#'].isin(ids_in_flights)

customers_without_flights = (
    data_customer.loc[mask_no_flights,
                      ['Loyalty#','Country','LoyaltyStatus','EnrollmentDateOpening','CancellationDate', COL_CLV]]
    .sort_values('Loyalty#')
    .reset_index(drop=True)
)

print(f"Customers in CustomerDB without any rows in FlightsDB: {len(customers_without_flights)}")
display(customers_without_flights.head(20))


Customers in CustomerDB without any rows in FlightsDB: 20


,Loyalty#,Country,LoyaltyStatus,EnrollmentDateOpening,CancellationDate,Customer Lifetime Value
0,100011,Canada,Star,5/1/2017,01/05/2017,NaN
1,100012,Canada,Star,2/27/2019,27/02/2019,NaN
2,100013,Canada,Star,9/20/2017,20/09/2017,NaN
3,100014,Canada,Star,11/28/2020,28/11/2020,NaN
4,100015,Canada,Star,4/9/2020,09/04/2020,NaN
5,100016,Canada,Star,7/21/2020,21/07/2020,NaN
6,100017,Canada,Star,4/11/2017,11/04/2017,NaN
7,999987,Canada,Star,3/7/2017,07/03/2017,NaN
8,999988,Canada,Star,8/22/2017,22/08/2017,NaN
9,999989,Canada,Star,9/12/2015,12/09/2015,NaN


In [1403]:
# Sanity check: confirm these customers have zero activity

target_ids = customers_without_flights['Loyalty#']

summary_no_flights = (
    data_flights.loc[data_flights['Loyalty#'].isin(target_ids)]
    .groupby('Loyalty#')
    .agg(
        flights=('NumFlights', 'sum'),
        distance_km=('DistanceKM', 'sum'),
        points_acc=('PointsAccumulated', 'sum'),
        points_red=('PointsRedeemed', 'sum'),
        redeemed_usd=('DollarCostPointsRedeemed', 'sum')
    )
    .reset_index()
)

print("Rows in the activity summary (should be 0):", len(summary_no_flights))
display(summary_no_flights.head())


Rows in the activity summary (should be 0): 0


,Loyalty#,flights,distance_km,points_acc,points_red,redeemed_usd


In [1404]:
# Set CLV = 0 ONLY for customers with:

mask_clv_nan = data_customer[COL_CLV].isna()
to_zero_mask = mask_no_flights & mask_clv_nan

ids_to_zero = data_customer.loc[to_zero_mask, 'Loyalty#']
print("Customers with CLV NaN & no flights:", len(ids_to_zero))
display(data_customer.loc[to_zero_mask, ['Loyalty#','Country','LoyaltyStatus']].head(20))

# Apply treatment
data_customer.loc[to_zero_mask, COL_CLV] = 0.0
print("Remaining CLV NaN:", data_customer[COL_CLV].isna().sum())


Customers with CLV NaN & no flights: 20


,Loyalty#,Country,LoyaltyStatus
16901,999987,Canada,Star
16902,999988,Canada,Star
16903,999989,Canada,Star
16904,999990,Canada,Star
16905,999991,Canada,Star
16906,999992,Canada,Star
16907,999993,Canada,Star
16908,999994,Canada,Star
16909,999995,Canada,Star
16910,999996,Canada,Star


Remaining CLV NaN: 0


In [1405]:
data_customer.isnull().sum()

Unnamed: 0                 0
Loyalty#                   0
First Name                 0
Last Name                  0
Customer Name              0
Country                    0
Province or State          0
City                       0
Latitude                   0
Longitude                  0
Postal code                0
Gender                     0
Education                  0
Location Code              0
Income                     0
Marital Status             0
LoyaltyStatus              0
EnrollmentDateOpening      0
CancellationDate           0
Customer Lifetime Value    0
EnrollmentType             0
dtype: int64

#### What we analysed & why CLV=0 for those 20 cases

**What we checked**
- We normalized the join key `Loyalty#` and performed an **anti-join** to find customers who are present in the CustomerDB but have **no rows** in the FlightsDB.
- For those customers, we verified that there is **no flight activity** at all (no flights, distance, points accumulated or redeemed, nor dollar redemptions).

**Findings**
- We identified **20 Canadian, Star-tier customers** who **enrolled and later cancelled** (based on `EnrollmentDateOpening` and `CancellationDate`) **without any recorded flight activity** during the observation window.
- Because CLV in this dataset is operationally tied to loyalty activity (points accumulated/redeemed and their dollar value), these customers have **no observable contribution** to CLV.

**Decision & rationale**
- We set `Customer Lifetime Value = 0` **only** for customers with **CLV missing** *and* **no flight activity**.
- This is a **transparent, conservative, and business-consistent** treatment: no activity → no realized (or observable) value.
- We deliberately avoided imputing a positive value (e.g., tier/country median) because that would fabricate value not supported by behavior, potentially biasing segmentation and KPIs.

**Notes**
- If a different business rule is preferred (e.g., imputing a peer median for reporting comparability), keep a flag and document it clearly as **imputed** rather than observed.
- All other customers retain their original CLV values.


All missing values 0

## Duplicates

In [1406]:
data_customer.duplicated().sum()

0

In [1407]:
data_flights.duplicated().sum()

2903

In [1408]:
# --- Exact duplicates: count, percent, and a quick peek ---

# total rows
n_total = len(data_flights)

# number of exact (full-row) duplicates
n_dupes = data_flights.duplicated().sum()

# % of exact duplicates
pct_dupes = (n_dupes / n_total * 100) if n_total else 0

print(f"Exact full-row duplicates: {n_dupes}  ({pct_dupes:.2f}%) out of {n_total} rows")

# Optional: preview all exact duplicates (no truncation)

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
 #   display(data_flights.loc[data_flights.duplicated(keep=False)]
  #          .sort_values(['Loyalty#','Year','Month']))


Exact full-row duplicates: 2903  (0.48%) out of 608436 rows


In [1409]:
dupe_patterns = (
    data_flights
      .value_counts()            
      .reset_index(name='count')
      .query('count > 1')
      .sort_values('count', ascending=False)
)
print(f"Distinct duplicated patterns: {len(dupe_patterns)}")
display(dupe_patterns.head(10))


Distinct duplicated patterns: 2875


,Loyalty#,Year,Month,YearMonthDate,NumFlights,NumFlightsWithCompanions,DistanceKM,PointsAccumulated,PointsRedeemed,DollarCostPointsRedeemed,count
0,678205,2019,3,3/1/2019,0.0,0.0,0.0,0.0,0.0,0.0,3
15,678205,2020,2,2/1/2020,0.0,0.0,0.0,0.0,0.0,0.0,3
1,678205,2020,4,4/1/2020,0.0,0.0,0.0,0.0,0.0,0.0,3
27,678205,2019,5,5/1/2019,0.0,0.0,0.0,0.0,0.0,0.0,3
26,678205,2021,12,12/1/2021,0.0,0.0,0.0,0.0,0.0,0.0,3
25,678205,2019,2,2/1/2019,0.0,0.0,0.0,0.0,0.0,0.0,3
23,678205,2019,1,1/1/2019,0.0,0.0,0.0,0.0,0.0,0.0,3
22,678205,2019,7,7/1/2019,0.0,0.0,0.0,0.0,0.0,0.0,3
21,678205,2019,8,8/1/2019,0.0,0.0,0.0,0.0,0.0,0.0,3
20,678205,2019,9,9/1/2019,0.0,0.0,0.0,0.0,0.0,0.0,3


In [1410]:
# Remove duplicates
data_flights = data_flights.drop_duplicates().copy()

In [1411]:
data_flights.duplicated().sum()

0

## Intigrity

### Foreign key integrity

In [1412]:
# Normalize join key as string
data_customer['Loyalty#'] = data_customer['Loyalty#'].astype(str).str.strip()
data_flights['Loyalty#']  = data_flights['Loyalty#'].astype(str).str.strip()

# Every flight Loyalty# must exist in customer
cust_ids = set(data_customer['Loyalty#'].unique())
fk_ok = data_flights['Loyalty#'].isin(cust_ids)

n_total = len(data_flights)
n_bad   = (~fk_ok).sum()
print(f"Flights with missing customer: {n_bad} ({n_bad/n_total:.2%})")

# Show a sample of offending rows (if any)
bad_rows = data_flights.loc[~fk_ok].copy()
display(bad_rows.head(20))


Flights with missing customer: 0 (0.00%)


,Loyalty#,Year,Month,YearMonthDate,NumFlights,NumFlightsWithCompanions,DistanceKM,PointsAccumulated,PointsRedeemed,DollarCostPointsRedeemed


In [1413]:
# Parse dates; your CancellationDate may also contain 'Active'
dc = data_customer.copy()

dc['enroll_dt'] = pd.to_datetime(dc['EnrollmentDateOpening'], errors='coerce', dayfirst=True)
# treat 'Active' as missing cancellation
dc['cancel_raw'] = dc['CancellationDate'].replace({'Active': pd.NA})
dc['cancel_dt']  = pd.to_datetime(dc['cancel_raw'], errors='coerce', dayfirst=True)

# Violations: cancellation exists and is earlier than enrollment
viol = dc.loc[dc['cancel_dt'].notna() & dc['enroll_dt'].notna() & (dc['cancel_dt'] < dc['enroll_dt'])]

print(f"Date violations (cancel < enroll): {len(viol)}")
display(viol[['Loyalty#','EnrollmentDateOpening','CancellationDate','enroll_dt','cancel_dt']].head(20))


Date violations (cancel < enroll): 202


,Loyalty#,EnrollmentDateOpening,CancellationDate,enroll_dt,cancel_dt
72,357549,9/21/2021,21/05/2019,2021-09-21,2019-05-21
77,265297,7/17/2021,17/03/2019,2021-07-17,2019-03-17
176,845613,10/13/2021,13/06/2019,2021-10-13,2019-06-13
210,830547,5/11/2021,11/01/2019,2021-05-11,2019-01-11
304,514900,7/20/2021,20/03/2019,2021-07-20,2019-03-20
556,381786,9/14/2021,14/05/2019,2021-09-14,2019-05-14
645,273353,7/18/2021,18/03/2019,2021-07-18,2019-03-18
678,698180,6/13/2021,13/02/2019,2021-06-13,2019-02-13
802,455287,7/18/2021,18/03/2019,2021-07-18,2019-03-18
881,456752,9/16/2021,16/05/2019,2021-09-16,2019-05-16


In [1414]:
# Aggregate per customer to compare totals
agg = (data_flights
       .groupby('Loyalty#', as_index=False)
       .agg(
           total_acc=('PointsAccumulated','sum'),
           total_red=('PointsRedeemed','sum'),
           total_red_usd=('DollarCostPointsRedeemed','sum')
       ))

# Negatives anywhere at row level (sanity)
neg_row_counts = (data_flights[['PointsAccumulated','PointsRedeemed','DollarCostPointsRedeemed']] < 0).sum()
print("Negative values at row level:\n", neg_row_counts.to_string())

# Customer-level violations
viol_points = agg.loc[
    (agg['total_red'] > agg['total_acc']) |
    (agg[['total_acc','total_red','total_red_usd']] < 0).any(axis=1)
].copy()

print(f"Customers with points violations: {len(viol_points)}")
display(viol_points.head(20))


Negative values at row level:
 PointsAccumulated           0
PointsRedeemed              0
DollarCostPointsRedeemed    0
Customers with points violations: 458


,Loyalty#,total_acc,total_red,total_red_usd
81,103975,5457.00,9014.0,90.0
120,106444,12108.00,12947.0,128.0
170,108857,8036.99,10549.9,103.2
174,109023,21905.53,25591.1,252.7
299,116238,6771.00,6798.0,67.0
308,116897,2664.00,10266.0,102.0
389,121236,10479.94,15830.8,155.8
396,121751,2828.00,3049.0,30.0
403,122020,42695.04,48555.3,477.9
409,122283,21994.82,24038.8,235.6
